# Week 3: Switches and Transistors

## Topics Covered:
- Relay Switches
- Limit Switches
- Diodes
- Transistors (BJT and MOSFET)
- Operational Amplifiers
- Light-Dependent Resistors (LDRs)

---

## Setup

In [ ]:
!pip install matplotlib numpy scipy schemdraw -q

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import schemdraw
import schemdraw.elements as elm
from scipy import signal

plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

## 1. Relay Switches

**Relay**: Electrically operated switch

### Key Features:
- Electromagnetic coil controls switch contacts
- Electrical isolation between control and load
- Can switch high voltage/current with low control signal
- Common types: SPST, SPDT, DPDT

### Applications:
- Home automation
- Industrial control
- Automotive systems
- Power switching

In [ ]:
# Relay timing simulation
time = np.linspace(0, 10, 1000)

# Control signal (5V microcontroller)
control_signal = np.zeros_like(time)
control_signal[(time >= 2) & (time < 7)] = 5

# Relay activation (with delay)
relay_state = np.zeros_like(time)
for i in range(len(time)):
    if control_signal[i] > 0:
        if i > 20:  # 20ms activation delay
            relay_state[i] = 1
    else:
        if i > 10:  # 10ms release delay
            relay_state[i] = 0

# Load state
load_voltage = relay_state * 120  # 120V AC load

fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12, 10))

ax1.plot(time, control_signal, 'b-', linewidth=2)
ax1.fill_between(time, 0, control_signal, alpha=0.3)
ax1.set_ylabel('Control (V)', fontsize=12)
ax1.set_title('Relay Control Signal (5V from MCU)', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3)

ax2.plot(time, relay_state, 'g-', linewidth=2)
ax2.fill_between(time, 0, relay_state, alpha=0.3)
ax2.set_ylabel('Relay State', fontsize=12)
ax2.set_title('Relay Contact State', fontsize=12, fontweight='bold')
ax2.set_yticks([0, 1])
ax2.set_yticklabels(['OPEN', 'CLOSED'])
ax2.grid(True, alpha=0.3)

ax3.plot(time, load_voltage, 'r-', linewidth=2)
ax3.fill_between(time, 0, load_voltage, alpha=0.3)
ax3.set_xlabel('Time (seconds)', fontsize=12)
ax3.set_ylabel('Load Voltage (V)', fontsize=12)
ax3.set_title('Load Voltage (120V AC)', fontsize=12, fontweight='bold')
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 2. Diodes

**Diode**: Semiconductor device that allows current in one direction only

### Types:
- **Standard Diode**: 1N4001-1N4007 series
- **LED**: Light-Emitting Diode
- **Zener**: Voltage regulation
- **Schottky**: Fast switching, low voltage drop

### Key Parameters:
- Forward Voltage Drop: ~0.7V (Si), ~0.3V (Schottky)
- Reverse Breakdown Voltage
- Maximum Forward Current

In [ ]:
# Diode I-V Characteristic
voltage = np.linspace(-10, 1, 500)

# Ideal diode model
I_s = 1e-12  # Saturation current
V_t = 0.026  # Thermal voltage at room temp
current = I_s * (np.exp(voltage / V_t) - 1)

# Limit current for visualization
current = np.clip(current * 1000, -0.1, 50)  # mA

plt.figure(figsize=(12, 6))

plt.plot(voltage, current, 'b-', linewidth=2)
plt.axhline(y=0, color='k', linestyle='-', linewidth=0.5)
plt.axvline(x=0, color='k', linestyle='-', linewidth=0.5)
plt.axvline(x=0.7, color='r', linestyle='--', alpha=0.5, label='Forward Voltage (0.7V)')

plt.xlabel('Voltage (V)', fontsize=12)
plt.ylabel('Current (mA)', fontsize=12)
plt.title('Diode I-V Characteristic Curve', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.xlim(-10, 1)
plt.ylim(-1, 50)

plt.tight_layout()
plt.show()

## 3. Transistors

**Transistor**: Semiconductor device for amplification and switching

### Types:

**BJT (Bipolar Junction Transistor)**:
- NPN and PNP types
- Current-controlled device
- Gain (β or hFE): Ic = β × Ib

**MOSFET (Metal-Oxide-Semiconductor FET)**:
- N-channel and P-channel
- Voltage-controlled device
- Very high input impedance
- Common: 2N7000, IRF540

In [ ]:
# BJT Switching Example
def bjt_switch_calculator(V_cc, R_load, V_be=0.7, beta=100, V_ce_sat=0.2):
    '''Calculate BJT switch parameters'''
    I_c = (V_cc - V_ce_sat) / R_load
    I_b = I_c / beta
    I_b_required = I_b * 2  # Use 2x for saturation
    
    print(f'BJT Switch Design')
    print(f'Supply Voltage: {V_cc}V')
    print(f'Load Resistance: {R_load}Ω')
    print(f'Beta (hFE): {beta}')
    print(f'\nCollector Current: {I_c*1000:.2f}mA')
    print(f'Base Current (min): {I_b*1000:.3f}mA')
    print(f'Base Current (recommended): {I_b_required*1000:.3f}mA')
    
    # Base resistor calculation (assuming 5V input)
    V_input = 5
    R_base = (V_input - V_be) / I_b_required
    print(f'\nBase Resistor (for {V_input}V input): {R_base:.0f}Ω')
    print(f'Standard value: {int(R_base/100)*100}Ω or {int(R_base/100+1)*100}Ω')
    
    return R_base

print('Example: LED driver with BJT')
bjt_switch_calculator(V_cc=9, R_load=1000, beta=100)

In [ ]:
# MOSFET vs BJT comparison
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# BJT characteristics
V_ce = np.linspace(0, 10, 100)
I_b_values = [10, 20, 30, 40, 50]  # µA
beta = 100

for I_b in I_b_values:
    I_c = beta * I_b / 1000  # mA
    I_c_curve = np.ones_like(V_ce) * I_c
    I_c_curve[V_ce < 0.2] = V_ce[V_ce < 0.2] * I_c / 0.2
    ax1.plot(V_ce, I_c_curve, linewidth=2, label=f'Ib={I_b}µA')

ax1.set_xlabel('VCE (V)', fontsize=12)
ax1.set_ylabel('IC (mA)', fontsize=12)
ax1.set_title('BJT Output Characteristics', fontsize=14, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# MOSFET characteristics
V_ds = np.linspace(0, 10, 100)
V_gs_values = [3, 4, 5, 6, 7]  # V

for V_gs in V_gs_values:
    if V_gs > 2.5:
        I_d = 0.1 * (V_gs - 2.5)**2  # Simplified model
        I_d_curve = np.ones_like(V_ds) * I_d
        I_d_curve[V_ds < 2] = I_d_curve[V_ds < 2] * V_ds[V_ds < 2] / 2
        ax2.plot(V_ds, I_d_curve, linewidth=2, label=f'Vgs={V_gs}V')

ax2.set_xlabel('VDS (V)', fontsize=12)
ax2.set_ylabel('ID (A)', fontsize=12)
ax2.set_title('MOSFET Output Characteristics', fontsize=14, fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Operational Amplifiers (Op-Amps)

**Op-Amp**: High-gain differential amplifier

### Key Characteristics:
- Very high input impedance
- Low output impedance
- High open-loop gain (>100,000)
- Virtual ground concept

### Common Configurations:
1. **Inverting Amplifier**: Gain = -Rf/Rin
2. **Non-Inverting Amplifier**: Gain = 1 + Rf/Rin
3. **Voltage Follower**: Gain = 1 (buffer)
4. **Comparator**: Compare two voltages

### Popular Op-Amps:
- LM358 (dual, general purpose)
- LM741 (classic, single)
- TL072 (dual, JFET input)
- LM324 (quad)

In [ ]:
def opamp_gain_calculator(R_in, R_f, inverting=True):
    '''Calculate op-amp gain'''
    if inverting:
        gain = -R_f / R_in
        config = 'Inverting'
    else:
        gain = 1 + R_f / R_in
        config = 'Non-Inverting'
    
    print(f'{config} Amplifier Configuration')
    print(f'Input Resistor (Rin): {R_in}Ω')
    print(f'Feedback Resistor (Rf): {R_f}Ω')
    print(f'Gain: {gain:.2f}x')
    print(f'Gain (dB): {20*np.log10(abs(gain)):.1f}dB')
    
    return gain

print('Example 1: Inverting amplifier (10x gain)')
opamp_gain_calculator(R_in=1000, R_f=10000, inverting=True)

print('\n' + '='*50 + '\n')

print('Example 2: Non-inverting amplifier (11x gain)')
opamp_gain_calculator(R_in=1000, R_f=10000, inverting=False)

In [ ]:
# Op-amp amplifier simulation
time = np.linspace(0, 0.02, 1000)  # 20ms
freq = 100  # Hz

# Input signal (100mV peak sine wave)
V_in = 0.1 * np.sin(2 * np.pi * freq * time)

# Amplified outputs
gain_inverting = -10
gain_noninverting = 10

V_out_inv = V_in * gain_inverting
V_out_noninv = V_in * gain_noninverting

fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12, 10))

ax1.plot(time * 1000, V_in * 1000, 'b-', linewidth=2)
ax1.axhline(y=0, color='k', linestyle='-', linewidth=0.5)
ax1.set_ylabel('Voltage (mV)', fontsize=12)
ax1.set_title('Input Signal (100mV peak)', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3)

ax2.plot(time * 1000, V_out_inv, 'r-', linewidth=2)
ax2.axhline(y=0, color='k', linestyle='-', linewidth=0.5)
ax2.set_ylabel('Voltage (V)', fontsize=12)
ax2.set_title('Inverting Amplifier Output (Gain = -10)', fontsize=12, fontweight='bold')
ax2.grid(True, alpha=0.3)

ax3.plot(time * 1000, V_out_noninv, 'g-', linewidth=2)
ax3.axhline(y=0, color='k', linestyle='-', linewidth=0.5)
ax3.set_xlabel('Time (ms)', fontsize=12)
ax3.set_ylabel('Voltage (V)', fontsize=12)
ax3.set_title('Non-Inverting Amplifier Output (Gain = 10)', fontsize=12, fontweight='bold')
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Light-Dependent Resistors (LDRs)

**LDR (Photoresistor)**: Resistance changes with light intensity

### Characteristics:
- **Dark Resistance**: 1MΩ - 10MΩ
- **Light Resistance**: 100Ω - 1kΩ
- Slow response time (~100ms)
- Wide spectral response

### Applications:
- Automatic lighting control
- Light meters
- Solar trackers
- Night lights

In [ ]:
# LDR behavior simulation
light_intensity = np.linspace(0, 1000, 100)  # Lux

# LDR resistance model (inverse relationship)
R_dark = 1000000  # 1MΩ
R_light = 100  # 100Ω at bright light
R_ldr = R_dark / (1 + light_intensity/10)

# Voltage divider output (10kΩ + LDR, 5V supply)
V_supply = 5
R_fixed = 10000
V_out = V_supply * R_ldr / (R_fixed + R_ldr)

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))

ax1.plot(light_intensity, R_ldr/1000, 'b-', linewidth=2)
ax1.set_xlabel('Light Intensity (Lux)', fontsize=12)
ax1.set_ylabel('LDR Resistance (kΩ)', fontsize=12)
ax1.set_title('LDR Resistance vs Light Intensity', fontsize=14, fontweight='bold')
ax1.set_yscale('log')
ax1.grid(True, alpha=0.3)

ax2.plot(light_intensity, V_out, 'g-', linewidth=2)
ax2.set_xlabel('Light Intensity (Lux)', fontsize=12)
ax2.set_ylabel('Output Voltage (V)', fontsize=12)
ax2.set_title('Voltage Divider Output (LDR + 10kΩ, 5V supply)', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3)
ax2.axhline(y=2.5, color='r', linestyle='--', alpha=0.5, label='Threshold (2.5V)')
ax2.legend()

plt.tight_layout()
plt.show()

## Practice Project: Light-Activated Relay

Design a circuit that turns on a relay when it gets dark.

### Components:
1. LDR + voltage divider
2. Op-amp comparator
3. Transistor driver
4. Relay

In [ ]:
# Simulate complete light-activated relay circuit
time = np.linspace(0, 24, 1000)  # 24 hours

# Simulate day/night cycle
light_level = 500 + 400 * np.sin(2 * np.pi * (time - 6) / 24)
light_level = np.maximum(light_level, 0)

# LDR voltage output
R_ldr_sim = 1000000 / (1 + light_level/10)
V_ldr = 5 * R_ldr_sim / (10000 + R_ldr_sim)

# Comparator output (threshold at 2.5V)
V_threshold = 2.5
comparator_out = (V_ldr > V_threshold).astype(int) * 5

# Relay state
relay_on = (V_ldr > V_threshold).astype(int)

fig, axes = plt.subplots(4, 1, figsize=(12, 12))

axes[0].plot(time, light_level, 'yellow', linewidth=2)
axes[0].fill_between(time, 0, light_level, alpha=0.3, color='yellow')
axes[0].set_ylabel('Light (Lux)', fontsize=12)
axes[0].set_title('Day/Night Light Cycle', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3)

axes[1].plot(time, V_ldr, 'b-', linewidth=2)
axes[1].axhline(y=V_threshold, color='r', linestyle='--', label=f'Threshold ({V_threshold}V)')
axes[1].set_ylabel('Voltage (V)', fontsize=12)
axes[1].set_title('LDR Voltage Output', fontsize=12, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

axes[2].plot(time, comparator_out, 'g-', linewidth=2)
axes[2].fill_between(time, 0, comparator_out, alpha=0.3)
axes[2].set_ylabel('Voltage (V)', fontsize=12)
axes[2].set_title('Comparator Output', fontsize=12, fontweight='bold')
axes[2].grid(True, alpha=0.3)

axes[3].plot(time, relay_on, 'r-', linewidth=2)
axes[3].fill_between(time, 0, relay_on, alpha=0.3, color='red')
axes[3].set_xlabel('Time (hours)', fontsize=12)
axes[3].set_ylabel('State', fontsize=12)
axes[3].set_title('Relay/Light State', fontsize=12, fontweight='bold')
axes[3].set_yticks([0, 1])
axes[3].set_yticklabels(['OFF', 'ON'])
axes[3].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print('Circuit operates automatically:')
print('- Daytime: LDR resistance low → Voltage high → Relay OFF → Light OFF')
print('- Nighttime: LDR resistance high → Voltage high → Relay ON → Light ON')

## Summary

This week we covered:

✅ **Relay Switches**: Electromagnetic switches for high-power loads  
✅ **Diodes**: One-way valves for current  
✅ **Transistors**: BJTs and MOSFETs for switching and amplification  
✅ **Op-Amps**: High-gain amplifiers for signal processing  
✅ **LDRs**: Light-sensitive resistors  
✅ **Project**: Light-activated relay circuit  

### Next Week Preview:
Week 4-5: Motors and Translators - Make things move! LAB 2: Robotic Arm